# Project ADA: Part 1 Read Data and Load DataFrame for all the Years

In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas==1.0.5

In [1]:
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import bz2
import json
from urllib.parse import urlparse
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import os
import glob
import pickle

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Create Dataframe for each journal 


In [2]:
def process_chunk(chunk, year, nb, df):
    
    df = chunk.drop(['qids', 'phase'], axis=1) #axis=1 for columns
    
    df['urls'] = df['urls'].astype('str') #Converting to string to be able to use str.contains 
    df_1 = df[df['urls'].str.contains('foxnews')] #Creating a DataFrame containing only foxnews
    df_2 = df[df['urls'].str.contains('nytimes')]
    df = pd.concat([df_1, df_2])

    df.to_pickle('/content/drive/MyDrive/ADA_2021/' + str(nb) + '_' +  str(year) + '_' + 'FoxNYtimes.pkl')

In [3]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
nb = 1
df = pd.DataFrame()
for y in years:
    nb = 1
    for chunk in pd.read_json('/content/drive/MyDrive/Quotebank/quotes-' + str(y) + '.json.bz2', 
                              lines=True, compression='bz2', chunksize=500000):
      
      process_chunk(chunk, y, nb, df)
      nb += 1

In [4]:
def read_yearly(y):
    PATH = '/content/drive/MyDrive/ADA_2021/Fox_NY_' + str(y) + '/'
    nb = 1
    df2 = []
    dirs = glob.glob(os.path.join(PATH, "*.pkl"))

    for files in dirs:
        df1 = pd.read_pickle(PATH + str(nb)+ '_' + str(y) + '_' + 'FoxNYtimes.pkl')
        df2.append(df1)
        nb += 1
    return df2

In [6]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
df_foxNy = pd.DataFrame()

for y in years:
  df_foxNy = df_foxNy.append(pd.concat(read_yearly(y)))

In [24]:
df_foxNy.sample(3)

,quoteID,quotation,speaker,date,numOccurrences,probas,urls
3708891,2019-12-31-004807,"At the end of the day, we're in a blessed situ...",Kyle Van Noy,2019-12-31 00:00:00,3,"[[Kyle Van Noy, 0.6539], [None, 0.3461]]",['http://www.foxnews.com/sports/patriots-kyle-...
13001237,2018-08-05-003781,Astronomy is virtually as popular in Africa as...,David Baratoux,2018-08-05 08:47:46,1,"[[David Baratoux, 0.8021], [None, 0.1862], [To...",['https://www.nytimes.com/2018/08/05/world/afr...
13789361,2016-09-26-030062,He's the king of kings! Seeing him on the stre...,Maryse Narcisse,2016-09-26 21:03:22,25,"[[Maryse Narcisse, 0.8254], [None, 0.1197], [J...",['http://www.seattletimes.com/nation-world/for...


## Checking if Filtering is Needed

In [12]:
#Searching for Missing Values
def MissingValuesFilter(chunk):
        #Gives True as a result, if the lines where NaN are presents are empty 
        #(meaning there are no NaNs):
        missing_nan = pd.DataFrame(np.where(chunk.isnull().any(axis=1))).empty
        #Gives True as a result if the lignes and corresponding columns where 
        #zeros, '' and None are found are empty:
        missing_zeros = pd.DataFrame(np.where(chunk==0)).empty 
        missing_space = pd.DataFrame(np.where(chunk=='')).empty
        missing_none = pd.DataFrame(np.where(chunk==None)).empty
        #missing_brackets = pd.DataFrame(np.where(chunk==[])).empty

        print('This DataFrame does not contain missing values')
        print(missing_nan, missing_zeros, missing_space, missing_none) 

MissingValuesFilter(df_foxNy)

/usr/local/lib/python3.7/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


This DataFrame does not contain missing values
True True True True


In [15]:
#Checking if Number of Occurrences is coherent
def NumOccurrencesFilter(chunk):
        size = len(chunk)
        #To have a number of occurrences equal zero has no meaning, so here it checks if this data set 
        if chunk['numOccurrences'].all() == np.array(size*[0]).all():
          print('Number of Occurrences impossible')
        else :
          print('Number of Occurences posible')

NumOccurrencesFilter(df_foxNy)

Number of Occurences posible


In [19]:
#Checking if the Highest Probability Corresponds to the Autor of the quotation
def ProbasFilter(chunk):
        size = len(chunk)
        if chunk['probas'].str[0].str[1].astype(float).all() < np.array(size*[0.8]).all():
          print('Probability of Autors of quotation too low')
        else :
          print('Probability of Autors higher than 80%')

ProbasFilter(df_foxNy)

Probability of Autors higher than 80%


In [20]:
#Checking if the Autor is the one with the highest probability of having generated the quotation
def ProbasFilter(chunk):  
        if chunk['speaker'].all() != chunk['probas'].str[0].str[0].all():
          print('The Autor does not correspond to the high probability Autor')
        else :
          print('The Autor has a high probability of having generated the quotation')

ProbasFilter(df_foxNy)

The Autor has a high probability of having generated the quotation
